In [ ]:
## Goes with the runCommandsOnNewCallback.yml file

In [ ]:
from mythic import mythic

In [ ]:
mythic_instance = await mythic.login(
        username="mythic_admin",
        password="mythic_password",
        server_ip="mythic_nginx",
        server_port=7443,
        timeout=-1
    )

In [ ]:
# ################ attributes available for env. in trigger:callback_new ################
## these are the fields you can access via env. in your Inputs. For example:
## env.display_id <-- that would give you the triggering callback's display_id field
callback_attribute_query = """
query MyQuery {
  callback(limit: 1) {
    active
    agent_callback_id
    architecture
    crypto_type
    current_time
    dead
    dec_key
    dec_key_base64
    description
    display_id
    domain
    enc_key
    enc_key_base64
    eventstepinstance_id
    external_ip
    extra_info
    host
    id
    init_callback
    integrity_level
    ip
    last_checkin
    locked
    locked_operator_id
    mythictree_groups
    mythictree_groups_string
    operation_id
    operator_id
    os
    pid
    process_name
    process_short_name
    registered_payload_id
    sleep_info
    timestamp
    user
  }
}
"""
fields = await mythic.execute_custom_query(
    mythic=mythic_instance,
    query=callback_attribute_query
)
print(fields)

In [ ]:
### Trigger Data

#trigger_data:
#  payload_types:
#    - poseidon
# This is how you limit this trigger to only happen on callbacks based on certain payload types

In [ ]:
### Keywords

#keywords:
#  - poseidon_callback

# keywords allow you to initiate a workflow by keyword and custom data instead of waiting for an event to happen
# in this case, if we use the keyword `poseidon_callback`, then we can trigger this workflow
# there's no requirement for these to be unique though, so you could trigger a bunch with a single keyword
# this triggering is available via the Scripting component:
trigger_keyword = """
mutation TriggerByKeyword($keyword: String!, $keywordEnvData: jsonb!){
    eventingTriggerKeyword(keyword: $keyword, keywordEnvData: $keywordEnvData){
        status
        error
    }
}
"""
triggerStatus = await mythic.execute_custom_query(
    mythic=mythic_instance,
    query=trigger_keyword,
    variables={"keyword": "poseidon_callback", "keywordEnvData": {"display_id": 1}}
)
# In this case, we are seeding this workflow's `display_id` to be 1, and providing no other data
# You can provide whatever data you want, but you should make sure that you provide everything that's needed by all your steps
print(triggerStatus)

In [ ]:
### action: task_create (string inputs)

# This works largely the same as how we issue tasks in scripting too, just in yaml form
#- name: "issue whoami"
#    description:
#    inputs:
#      CALLBACK_ID: env.display_id <-- notice this gets the display_id of the callback that triggered this event
#      COMMAND: shell <-- this could be hardcoded in the action_data, but left here as a simple example
#    action: task_create
#    action_data: <-- the actual data associated with this action, task_create
#      callback_display_id: CALLBACK_ID <-- this gets replaced based on our inputs
#      params: whoami
#      command_name: COMMAND <-- this gets replaced based on our inputs

In [ ]:
### action: task_create (but with a continue_on_error)

#  - name: "getenv"
#    description:
#    inputs:
#      CALLBACK_ID: env.display_id
#    action: task_create
#    action_data:
#      callback_display_id: CALLBACK_ID
#      params: 
#      command_name: getenv
#    continue_on_error: true <-- normally, if one step errors, then the entire workflow is cancelled
#                       ^ this is how you can say that if this step errors, still continue on with the rest of the workflow

In [ ]:
### action: task_create (with workflow files and dictionaries)

#  - name: "import script"
#    description: "import the HealthInspector script into the callback"
#    inputs:
#      CALLBACK_ID: env.display_id <-- env.display_id gets the display_id from the triggering callback
#      FILE_ID: workflow.HealthInspector.js <-- workflow.* gets the agent_file_id associated with files uploaded as part of the workflow
#               ^ click the paperclip icon when viewing a workflow to upload / manage files to use with it
#    action: task_create
#    action_data:
#      callback_display_id: CALLBACK_ID <-- repalced based on our inputs
#      params_dictionary: <-- notice this is params_dictionary instead of just params, now we can provide non-ambiguous parameters 
#        file_id: FILE_ID <-- replaced based on our inputs
#      command_name: jsimport

In [ ]:
### action: task_create (depends_on and outputs)

# - name: "run script"
#    description: "run the HealthInspector script"
#    inputs:
#      CALLBACK_ID: env.display_id
#    action: task_create
#    action_data:
#      callback_display_id: CALLBACK_ID
#      params_dictionary:
#        filename: HealthInspector.js <-- this specific command looks up files by name instead of agent_file_id, so we can just specify the name
#        code: All_Checks(); <-- the code we want to run
#      command_name: jsimport_call
#    depends_on:
#      - import script <-- the "name" field from the task that imported the file. This task won't run until that task executes successfully
#    outputs:
#      SCRIPT_TASK_ID: id <-- this step created a task, so we can save off any attribute of the task as output, all options shown below

In [ ]:
task_attribute_query = """
query MyQuery {
  task(limit: 1) {
    agent_task_id
    apitokens_id
    callback_id
    command_id
    command_name
    comment
    comment_operator_id
    completed
    completed_callback_function
    completed_callback_function_completed
    display_id
    display_params
    eventstepinstance_id
    group_callback_function
    group_callback_function_completed
    has_intercepted_response
    id
    interactive_task_type
    is_interactive_task
    operation_id
    operator_id
    opsec_post_blocked
    opsec_post_bypass_role
    opsec_post_bypass_user_id
    opsec_post_bypassed
    opsec_post_message
    opsec_pre_blocked
    opsec_pre_bypass_role
    opsec_pre_bypass_user_id
    opsec_pre_bypassed
    opsec_pre_message
    original_params
    parameter_group_name
    params
    parent_task_id
    response_count
    status
    status_timestamp_preprocessing
    status_timestamp_processed
    status_timestamp_processing
    status_timestamp_submitted
    stderr
    stdout
    subtask_callback_function
    subtask_callback_function_completed
    subtask_group_name
    tasking_location
    timestamp
    token_id
  }
}
"""
fields = await mythic.execute_custom_query(
    mythic=mythic_instance,
    query=task_attribute_query
)
print(fields)

In [ ]:
### action: custom_function (apitokens, custom functions, and inputs from other tasks)

#  - name: "check for EDR"
#    description: "Process the output of the HealthInspector task to see if there's any EDR running"
#    inputs:
#      SCRIPT_TASK_ID: "run script.SCRIPT_TASK_ID" <-- one of our inputs will be this named output from a previous step
#      APIToken: mythic.apitoken <-- we can ask Mythic for a custom APIToken that'll track what we do to this specific step
#      CALLBACK_ID: env.display_id <-- still get the same triggering callback's display_id
#    action: custom_function
#    action_data:
#      function_name: HealthInspectorOutputEDRCheck <-- the name of the CustomFunction entry within our container to call
#      container_name: opsecChecker <-- the name of the custom container we created that has the above function for us to call
#    depends_on:
#      - run script <-- the "name" field from the other step that needs to complete first

In [ ]:
### Other thoughts
# If you just load the runCommandsOnNewCallback.yml into Mythic you'll see that there's a red warning about missing the specified container
# Mythic won't currently alert you that you specify a workflow file that isn't upload (future update)
